In [1]:
# -*- coding: utf-8 -*-
import torch
import torch.optim as optim

In [6]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigvalsh 

## Inverse problem data

In [20]:
# Physical data
l = 200
tau = 100
dep = 2
# Numerical data
nx = 200
dx = l/(nx+1)
nt = 200
dt = tau/nt
T_operator = 1/100*dx*np.tri(nt, nx, 0, dtype=int)

In [24]:
# training data
ntrain = 400
x_dagger = np.zeros((ntrain,nx))
x_init = np.zeros((ntrain,nx))
y = np.zeros((ntrain,nt))
x_grid = np.linspace(0,l,nx)
#
x_train = np.zeros((ntrain,nt))
#
for i in range(0,ntrain):
    # value for x
    mu = l/2
    sigma = 0.1
    x_dagger[i,:] = (sigma*np.sqrt(2*np.pi))**-1*np.exp(-(x_grid-mu)**2/2*sigma**2)
    y[i,:] = T_operator.dot(x_dagger[i,:]) 
    xi = np.random.uniform(-0.05,0.05,nt)
    y[i,:] += 0.05*xi*np.linalg.norm(y[i,:])/np.linalg.norm(xi)
    x_train[i,:] = np.transpose(T_operator).dot(y[i,:])

## Autograd function

In [25]:

class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input



In [36]:
dtype = torch.float
device = torch.device("cpu")

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 400, 200, 200, 200

# Create random Tensors to hold input and outputs.
x = torch.tensor(x_train, dtype=torch.float)
y = torch.tensor(x_dagger, dtype=torch.float)

# Create Tensors and coeff of weights.
coeff = torch.tensor(0.00005, dtype=torch.float, requires_grad=True)
w0 = torch.tensor(np.eye(nx), dtype=torch.float, requires_grad=False)
w1 = torch.tensor(np.transpose(T_operator).dot(T_operator), dtype=torch.float, requires_grad=False)
w01 = w0 + coeff*w1
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)


In [37]:
learning_rate = 0.00001
for t in range(500):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    relu = MyReLU.apply

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = relu(x.mm(w01)).mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward(retain_graph=True)

    # Update weights using gradient descent
    with torch.no_grad():
        coeff -= learning_rate * coeff.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        coeff.grad.zero_()
        w2.grad.zero_()

99 35.94449234008789
199 35.93788528442383
299 35.931209564208984
399 35.924625396728516
499 35.9179573059082
